#Aplicando el modelo de regresión logística y sus respectivas metricas
En este dataset tratare de hacer un buen analisis y tratamiento de los datos para poder obtener un buen resultado usando la regresión logistica y sus respectivas metricas como lo son la matriz de confusion o su curva roc, esto para poner en practica conceptos aprendidos, un buen ejemplo podria ser un dataset de deteccion de fraude a lo cual nos servira para motrar las fortalezas y debilidades del algoritmo

##Importación de las librerias necesarias
Aqui ire importando todas las librerias que requiera a lo largo de este notebook, por el momento estaran las basicas e ire añadiendo más

In [3]:
!pip install opendatasets
import opendatasets as od
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

#Dataset
Para este caso como comente trabajaremos con un datset llamado "Fraudulent Transaction Detection" donde a base de este dataset detectaremos si una transacción es fraudulenta o no, y es uno de los primeros datsets con una buena cantidad de datos.

##Vocabulario del dataset
* TRANSACTION_ID = Unique identifier for the transaction
* TX_DATETIME = Date and time at which the transaction occurs
* CUSTOMER_ID = Unique identifier for the customer
* TERMINAL_ID = Unique identifier for the terminal through which transaction took place
* TX_AMOUNT = Amount of Transaction
* X_TIME_SECONDS = Seconds elapsed before the transaction
* TX_TIME_DAYS = Seconds elapsed before the transaction
* TX_FRAUD = A binary variable, with the value 0 for a legitimate transaction, or the value 1 for a fraudulent transaction.
* TX_FRAUD_SCENARIO = A ternary variable depicting different fraudulent scenarios



In [2]:
datasetlink = "https://www.kaggle.com/datasets/sanskar457/fraud-transaction-detection"
od.download(datasetlink)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: kidkeozz
Your Kaggle Key: ··········


100%|██████████| 34.0M/34.0M [00:01<00:00, 28.6MB/s]


In [5]:
os.chdir("fraud-transaction-detection")
os.listdir()

['Final Transactions.csv']

In [19]:
#Damos un primer vistazo a los datos
archivo = 'Final Transactions.csv'
df = pd.read_csv(archivo)
df.head()

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,0,2023-01-01 00:00:31,596,3156,533.07,31,0,0,0
1,1,1,2023-01-01 00:02:10,4961,3412,808.56,130,0,0,0
2,2,2,2023-01-01 00:07:56,2,1365,1442.94,476,0,1,1
3,3,3,2023-01-01 00:09:29,4128,8737,620.65,569,0,0,0
4,4,4,2023-01-01 00:10:34,927,9906,490.66,634,0,0,0


In [7]:
#dimension del dataset cuenta con 1754155 datos y 10 columnas
df.shape

(1754155, 10)

In [9]:
#Obtenemos más información acerca de la estructura del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754155 entries, 0 to 1754154
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   TRANSACTION_ID     int64  
 2   TX_DATETIME        object 
 3   CUSTOMER_ID        int64  
 4   TERMINAL_ID        int64  
 5   TX_AMOUNT          float64
 6   TX_TIME_SECONDS    int64  
 7   TX_TIME_DAYS       int64  
 8   TX_FRAUD           int64  
 9   TX_FRAUD_SCENARIO  int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 133.8+ MB


In [10]:
df.describe()

,Unnamed: 0,TRANSACTION_ID,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
count,1.754155e+06,1.754155e+06,1.754155e+06,1.754155e+06,1.754155e+06,1.754155e+06,1.754155e+06,1.754155e+06,1.754155e+06
mean,8.770770e+05,8.770770e+05,2.504011e+03,4.996733e+03,5.396820e+02,7.903234e+06,9.097260e+01,1.345200e-01,1.449746e-01
std,5.063811e+05,5.063811e+05,1.445987e+03,2.886101e+03,1.179711e+03,4.565172e+06,5.283709e+01,3.412103e-01,3.874872e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.385385e+05,4.385385e+05,1.252000e+03,2.502000e+03,1.799500e+02,3.940846e+06,4.500000e+01,0.000000e+00,0.000000e+00
50%,8.770770e+05,8.770770e+05,2.506000e+03,4.994000e+03,4.217500e+02,7.902670e+06,9.100000e+01,0.000000e+00,0.000000e+00
75%,1.315616e+06,1.315616e+06,3.765000e+03,7.495000e+03,7.505950e+02,1.186566e+07,1.370000e+02,0.000000e+00,0.000000e+00
max,1.754154e+06,1.754154e+06,4.999000e+03,9.999000e+03,6.478375e+05,1.581120e+07,1.820000e+02,1.000000e+00,3.000000e+00


In [11]:
df.isnull().sum()

Unnamed: 0           0
TRANSACTION_ID       0
TX_DATETIME          0
CUSTOMER_ID          0
TERMINAL_ID          0
TX_AMOUNT            0
TX_TIME_SECONDS      0
TX_TIME_DAYS         0
TX_FRAUD             0
TX_FRAUD_SCENARIO    0
dtype: int64

In [20]:
df = df.drop(columns=['Unnamed: 0','TRANSACTION_ID'])

In [21]:
df

,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,2023-01-01 00:00:31,596,3156,533.07,31,0,0,0
1,2023-01-01 00:02:10,4961,3412,808.56,130,0,0,0
2,2023-01-01 00:07:56,2,1365,1442.94,476,0,1,1
3,2023-01-01 00:09:29,4128,8737,620.65,569,0,0,0
4,2023-01-01 00:10:34,927,9906,490.66,634,0,0,0
...,...,...,...,...,...,...,...,...
1754150,2023-07-02 23:56:36,161,655,531.43,15810996,182,0,0
1754151,2023-07-02 23:57:38,4342,6181,10.40,15811058,182,0,0
1754152,2023-07-02 23:58:21,618,1502,26.62,15811101,182,0,0
1754153,2023-07-02 23:59:52,4056,3067,550.43,15811192,182,0,0


##Primer problemática
como vemos en la linea de abajo tenemos 222261 transacciones marcadas como fraudulentas esto puede sonar a bastantes y lo son, pero si lo comparamos con la cantidad de datos que tenemos esto puede representar un problema ya que hay un desbalance en las etiquetas de si es fraude o no, por lo cual esto representa nuestro primer problema, sin embargo para todo problema existe una solución o al menos para este caso puede ser asi, asi que buscaremos un procedimiento que iremos explicando en las celdas de más abajo.

In [17]:
df[df['TX_FRAUD_SCENARIO']==1].shape

(222261, 9)